In [234]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from langdetect import detect
import time
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.model_selection import train_test_split
import numpy as np
import json

import tensorflow as tf
from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors
import pickle

import matplotlib.pyplot as plt
import joblib
import random
import nltk, re, time
from nltk.corpus import stopwords
import string
from collections import defaultdict
from tqdm import tqdm

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.utils import to_categorical
from collections import namedtuple

from contractions import get_contractions
import operator
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import tensor_array_ops
print('TensorFlow Version: {}'.format(tf.__version__))

alreadyPickled = False

TensorFlow Version: 1.4.0


## Variable Initialization

In [279]:
embedding_dim = 300
num_layers = 2
num_classes = 6
epochs = 100
batch_size = 5
rnn_size = 128
num_layers = 2

learning_rate = 0.005

keep_probability = 0.75
max_sequence_length = 1000


In [248]:
def clean_text(text, remove_stopwords = True):
    assert isinstance(text, str)
    text = text.lower()    
    if True:
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    # Format words and remove unwanted characters
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text,  
                  flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    # Optionally, remove stop words
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)
    return text

In [18]:
restId = []
for line in open('./data/dataset/business.json', 'r'):
    data = json.loads(line)
    if 'Restaurants' in data['categories'] or 'Food' in data['categories']:
        restId.append(data['business_id'])
print("Done getting Business IDs")
print("There are %d restaurants" % (len(restId)))

Done getting Business IDs
There are 65028 restaurants


In [21]:
contractions = get_contractions()

revs_list = [[]]
stars_list = [[]]
num = 5000 # Number of review read
k = 0 # Count
for line in open('./data/dataset/review.json', 'r'):
    if k >= num:
        break
    data = json.loads(line)
    text = data['text']
    star = data['stars']
    ID = data['business_id']
    # Check language
    if detect(text) != 'en':
        continue  
    # Check whether it is a restaurant
    if ID not in restId:
        continue
    revs_list.append(clean_text(text))
    stars_list.append(star)
    k += 1
    # Notify for every 500 reviews
    if len(revs_list) % 500 == 0:
        print(len(revs_list))

500
1000
1500
2000
2500
3000
3500
4000
4500
5000


In [249]:
print(revs_list[1])
print(len(revs_list), len(stars_list))

place horrible excited try since got gift card birthday went ordered whole meal except gift card system unacceptable would helpful would known prior
5001 5001


In [250]:
zipped_revs = zip(revs_list, stars_list)
print("Shape of Ziiped Revs is [%d, %d]" % (len(zipped_revs), len(zipped_revs[0])))
categories = ['text', 'stars']
df_reviews_processing = pd.DataFrame(zipped_revs, columns=categories)


Shape of Ziiped Revs is [5001, 2]


In [251]:
df_reviews_processing.shape
df_reviews_processing.head()
df_reviews_processing[['stars']] = df_reviews_processing[['stars']].apply(pd.to_numeric)

df_reviews_processing = df_reviews_processing[np.isfinite(df_reviews_processing['stars'])]

print("Before Dropping Nones: Shape %d,%d" % (df_reviews_processing.shape[0], df_reviews_processing.shape[1]))
df_reviews = df_reviews_processing[np.isfinite(df_reviews_processing['stars'])]
print("After Dropping Nones: Shape %d,%d" % (df_reviews_processing.shape[0], df_reviews_processing.shape[1]))

df_reviews_processing.to_csv("reviews_df_processed.csv", encoding='utf-8')



Before Dropping Nones: Shape 5000,2
After Dropping Nones: Shape 5000,2


# Data Loading
If the data is already pickled, then can skip embedding and data processing

In [252]:
def picklefiles(filename, stuff):
    save_stuff = open(filename, "wb")
    pickle.dump(stuff, save_stuff)
    save_stuff.close()
def loadfiles(filename):
    saved_stuff = open(filename,"rb")
    stuff = pickle.load(saved_stuff)
    saved_stuff.close()
    return stuff

## Embeddings
Using ConceptNet Numberbatch instead of GLoVE (supposedly outperforms GLoVE embeddings)  
  
  
On top of the embeddings, we also keep track of commonly used words in the reviews that Embeddings don't cover. This way we could have higher test accuracy when words we come across words like these. This is specified by a threshold value. Currently, threshold is set to 20 occuraces.  
  
  
We also process the reviews a bit more, sorting them into comparable lengths. This way, there is less padding necessary and (possibly) faster computation time when training/testing

In [253]:
def count_words(count_dict, text):
    '''Count the number of occurrences of each word in a set of text'''
    for sentence in text:
        for word in sentence.split():
            if word not in count_dict:
                count_dict[word] = 1
            else:
                count_dict[word] += 1

In [254]:
word_counts = {}
count_words(word_counts, df_reviews.text)            
print("Size of Vocabulary:", len(word_counts))

Size of Vocabulary: 15259


In [ ]:
embed_path='./embeddings/numberbatch-en-17.02.txt'
def load_embeddings(path='./embeddings/numberbatch-en-17.02.txt'):
    embeddings_index = {}
    with open(path) as f:
        for line in f:
            values = line.split(' ')
            word = values[0]
            embedding = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = embedding
    return embeddings_index

embeddings_index = load_embeddings(embed_path)

# Balance the Dataset
Want to balance the dataset, such that we have an equal number of reviews for each different category.  
For example, if our distribution of reviews is [200,500,100,300,400], for [1,2,3,4,5] stars, respectively, then we will only take 100 of each review

In [255]:
def balance_dataframe(df, category=['stars'], downsample_by=0.9):
    """
    :param df: pandas.DataFrame
    :param categorical_columns: iterable of categorical columns names contained in {df}
    :return: balanced pandas.DataFrame
    """
    if category is None or not all([col in df.columns for col in category]):
        raise ValueError('Please provide one or more columns containing categorical variables')

    lowest_count = df.groupby(category).apply(lambda x: x.shape[0]).min()
    # Remove 10 percent of the reviews
    lowest_count = int(lowest_count * downsample_by)
    
    df = df.groupby(category).apply( 
        lambda x: x.sample(lowest_count)).drop(category, axis=1).reset_index().set_index('level_1')

    df.sort_index(inplace=True)

    return df

In [256]:
# df_reviews = pd.read_csv('reviewsText.csv')
df_reviews = pd.read_csv("reviews_df_processed.csv")
df_reviews['len'] = df_reviews.text.str.len()
df_reviews = df_reviews[df_reviews['len'].between(10, 4000)]


print("(pre) 1 star ratings: %d" % (len(df_reviews[df_reviews.stars == 1])))
print("(pre) 2 star ratings: %d" % (len(df_reviews[df_reviews.stars == 2])))
print("(pre) 3 star ratings: %d" % (len(df_reviews[df_reviews.stars == 3])))
print("(pre) 4 star ratings: %d" % (len(df_reviews[df_reviews.stars == 4])))
print("(pre) 5 star ratings: %d" % (len(df_reviews[df_reviews.stars == 5])))

# Set downsample_by to 0.9, since not much training data to begin with
df_balanced = balance_dataframe(df_reviews, 
                                category=['stars'], 
                                downsample_by=0.9)

df_balanced.to_csv('balanced_reviews1000.csv', encoding='utf-8')
print("(post) 1 star ratings: %d" % (len(df_balanced[df_balanced.stars == 1])))
print("(post) 2 star ratings: %d" % (len(df_balanced[df_balanced.stars == 2])))
print("(post) 3 star ratings: %d" % (len(df_balanced[df_balanced.stars == 3])))
print("(post) 4 star ratings: %d" % (len(df_balanced[df_balanced.stars == 4])))
print("(post) 5 star ratings: %d" % (len(df_balanced[df_balanced.stars == 5])))




(pre) 1 star ratings: 469
(pre) 2 star ratings: 545
(pre) 3 star ratings: 759
(pre) 4 star ratings: 1353
(pre) 5 star ratings: 1874
(post) 1 star ratings: 422
(post) 2 star ratings: 422
(post) 3 star ratings: 422
(post) 4 star ratings: 422
(post) 5 star ratings: 422


In [211]:
print(1 in df_reviews['stars'])
print(2 in df_reviews['stars'])
print(3 in df_reviews['stars'])
print(4 in df_reviews['stars'])
print(5 in df_reviews['stars'])

True
True
True
True
True


In [257]:
embedding_dim = 300
MAX_NB_WORDS = 20000

tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(text)

joblib.dump(tokenizer, 'tokenizer.p')

WORD_INDEX_SORTED = sorted(tokenizer.word_index.items(), key=operator.itemgetter(1))
print("number of words is %d" % (len(WORD_INDEX_SORTED)))

vocab_size = len(WORD_INDEX_SORTED)+1

NB_WORDS = min(len(WORD_INDEX_SORTED), MAX_NB_WORDS)

word_embedding_matrix = np.zeros((NB_WORDS, embedding_dim), dtype=np.float32)

for word, i in tokenizer.word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        word_embedding_matrix[i-1] = embedding_vector
print("embedding matrix size: is [%d,%d]" % (word_embedding_matrix.shape[0], 
                                            word_embedding_matrix.shape[1]))

number of words is 23
embedding matrix size: is [23,300]


In [258]:
seqs = tokenizer.texts_to_sequences(df_balanced.text.values)

padReviews = pad_sequences(seqs, maxlen=max_sequence_length)

reviewsLength = max_sequence_length

ratings = df_balanced.stars.values.astype(int)
ratings_cat = to_categorical(ratings)

X_train, X_test, y_train, y_test = train_test_split(padReviews, ratings_cat, test_size=0.2, random_state=9)
with pd.HDFStore('x_y_test_train.h5') as h:
    h['X_train'] = pd.DataFrame(X_train)
    h['X_test'] = pd.DataFrame(X_test)
    h['y_train'] = pd.DataFrame(y_train)
    h['y_test'] = pd.DataFrame(y_test)

assert padReviews.shape[0] == ratings.shape[0] 

In [259]:
print(type(ratings_cat))
print(ratings_cat.shape)
print(X_train.shape)
print(y_train.shape)

<type 'numpy.ndarray'>
(2110, 6)
(1688, 1000)
(1688, 6)


In [111]:
if alreadyPickled == False:
    picklefiles("./data/good_pickles/balanced_reviews.p",df_balanced)
    picklefiles("./data/good_pickles/category_ratings.p",ratings_cat)
    picklefiles("./data/good_pickles/word_embedding_matrix.p",word_embedding_matrix)
    
if alreadyPickled == True:

    word_embedding_matrix = loadfiles("./data/word_embedding_matrix.p")
    df_rev_balanced = pd.read_csv('balanced_reviews.csv')
    
    ratings_cat = loadfiles("./data/good_pickles/category_ratings.p")
    balanced_reviews = loadfiles("./data/good_pickles/balanced_reviews.p")
    
    tokenizer = joblib.load('tokenizer.pickle')
    with pd.HDFStore('x_y_test_train.h5') as h:
        X_train = h['X_train'].values
        X_test = h['X_test'].values
        y_train = h['y_train'].values
        y_test = h['y_test'].values

# Gettin Batches
Gets batches as well as pads them to have similar length

In [262]:
def get_batches(ratings, reviews, batch_size):
    """Batch summaries, texts, and the lengths of their sentences together"""
    for batch_i in range(0, len(reviews)//batch_size):
        start_i = batch_i * batch_size
        rating_batch = ratings[start_i:start_i + batch_size, :]
        reviews_batch = reviews[start_i:start_i + batch_size, :]

        yield rating_batch, reviews_batch

In [263]:
for batch_i, (trainX_batch, trainY_batch) in enumerate(get_batches(X_train, y_train, batch_size)):
    print(trainX_batch.shape)
    print(trainY_batch.shape)
    if batch_i == 5:
        break

(5, 1000)
(5, 6)
(5, 1000)
(5, 6)
(5, 1000)
(5, 6)
(5, 1000)
(5, 6)
(5, 1000)
(5, 6)
(5, 1000)
(5, 6)


# Building the Graph

In [272]:
def model_inputs():

    # Should be [batch_size x review length]
    input_data = tf.placeholder(tf.int32, [None, None], name='input')
    # Should be [batch_size x num_classes]
    labels = tf.placeholder(tf.int32, [None, None], name='labels')
    
    lr = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

    return input_data, labels, lr, keep_prob

In [276]:
tf.reset_default_graph()

In [277]:
train_graph = tf.Graph()
with train_graph.as_default():
    
    input_data, labels, lr, keep_prob = model_inputs()
    
    print("Shape of Input Data")
    print(tf.shape(input_data))
    embeddings = word_embedding_matrix
    embs = tf.nn.embedding_lookup(embeddings, input_data)

    W2 = tf.Variable(np.random.rand(rnn_size, num_classes),dtype=tf.float32)
    b2 = tf.Variable(np.zeros((1, num_classes)), dtype=tf.float32)


    with tf.name_scope("RNN_Layers"):
        stacked_rnn = []
        for layer in range(num_layers):
            cell_fw = tf.contrib.rnn.GRUCell(rnn_size)
            cell_fw = tf.contrib.rnn.DropoutWrapper(cell_fw,
                                                    output_keep_prob=keep_prob)
            stacked_rnn.append(cell_fw)
        multilayer_cell = tf.contrib.rnn.MultiRNNCell(stacked_rnn, state_is_tuple=True)

    with tf.name_scope("init_state"):
        initial_state = multilayer_cell.zero_state(batch_size, tf.float32)

    with tf.name_scope("Forward_Pass"):
        output, states_series = tf.nn.dynamic_rnn(multilayer_cell,
                                           embs,
                                           dtype=tf.float32)

    states_s = tf.reshape(states_series, [-1, rnn_size])

    logits = tf.matmul(states_s, W2) + b2 #Broadcasted addition
    # labels = tf.reshape(labels, [-1])

    logits_series = tf.unstack(tf.reshape(logits, [batch_size, num_classes]), axis=1)
    predictions_series = [tf.nn.softmax(logit) for logit in logits_series]

    with tf.name_scope("loss"):
#         dense_labels = tf.sparse_to_dense(labels)
        losses = tf.losses.softmax_cross_entropy(logits=logits, onehot_labels=labels)

        total_loss = tf.reduce_mean(losses)
        tf.summary.scalar("cost", total_loss)
        
    with tf.name_scope("optimization"):
        # Optimizer
        train_step = tf.train.RMSPropOptimizer(learning_rate).minimize(total_loss)
#         gradients = optimizer.compute_gradients(total_loss)
#         capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
#         train_op = optimizer.apply_gradients(capped_gradients)


print("Graph is built.")
graph_location = "./graph"
print(graph_location)
train_writer = tf.summary.FileWriter(graph_location)
train_writer.add_graph(train_graph)
    # with tf.name_scope("FC_Layer"):
    #     dense = tf.contrib.layers.fully_connected(output[:,-1],
    #                                               num_outputs = 64,
    #                                               activation_fn = tf.sigmoid,
    #                                               weights_initializer = W2,
    #                                               biases_initializer = b2)
    #     dense = tf.contrib.layers.dropout(dense, keep_prob)

    # with tf.name_scope("Predictions"):
    #     predictions = tf.contrib.layers.fully_connected(dense,
    #                                                     num_outputs = num_classes, 
    #                                                     activation_fn=tf.nn.relu,
    #                                                     weights_initializer = W2,
    #                                                     biases_initializer = b2)

    # states_series_reshaped = tf.reshape(states_series, [-1, rnn_size])

    # logits = tf.matmul(states_series_reshaped, W2) + b2 #Broadcasted addition
    # labels_logits = tf.reshape(labels, [-1])


    # logits_series = tf.unstack(tf.reshape(logits, [batch_size, num_classes]), axis=1)
    # predictions_series = [tf.nn.softmax(logit) for logit in logits_series]

Shape of Input Data
Tensor("Shape:0", shape=(2,), dtype=int32)
Graph is built.
./graph


In [243]:
print(tf.shape(output))


Tensor("Shape_13:0", shape=(3,), dtype=int32)


In [244]:
print(tf.shape(logits))

Tensor("Shape_14:0", shape=(2,), dtype=int32)


# Training the Model


In [269]:
learning_rate_decay = 0.95
min_learning_rate = 0.0005
display_step = 20 # Check training loss after every 20 batches
stop_early = 0 
stop = 3 # If the update loss does not decrease in 3 consecutive update checks, stop training
per_epoch = 3 # Make 3 update checks per epoch
update_check = (len(X_train)//batch_size//per_epoch)-1
keep_probability = 0.75
update_loss = 0 
batch_loss = 0
summary_update_loss = [] # Record the update losses for saving improvements in the model

In [278]:
checkpoint = "./saves/best_model.ckpt" 
logs_path = "./logs/"
summary_ops = tf.summary.merge_all()

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
    ###
    # If we want to continue training a previous session
    ###
#     loader = tf.train.import_meta_graph("./" + checkpoint + '.meta')
#     loader.restore(sess, checkpoint)

    
    for epoch_i in range(1, epochs+1):
        update_loss = 0
        batch_loss = 0
        for batch_i, (trainX_batch, trainY_batch) in enumerate(get_batches(X_train, y_train, batch_size)):
            if batch_i < 10:
                print(trainX_batch)
                print(trainY_batch)
            start_time = time.time()
            _, loss= sess.run([train_step, total_loss],
                              feed_dict={
                                  input_data: trainX_batch,
                                  labels: trainY_batch,
                                  lr: learning_rate,
                                  keep_prob: keep_probability})
            
            
#             writer.add_summary(summary, epoch_i * batch_size + batch_i)
            
            batch_loss += loss
            update_loss += loss
            end_time = time.time()
            batch_time = end_time - start_time

            if batch_i % display_step == 0 and batch_i > 0:
                print('Epoch {:>3}/{} Batch {:>4}/{} - Loss: {:>6.3f}, Seconds: {:>4.2f}'
                      .format(epoch_i,
                              epochs, 
                              batch_i, 
                              len(X_train) // batch_size, 
                              batch_loss / display_step, 
                              batch_time*display_step))
                batch_loss = 0

            if batch_i % update_check == 0 and batch_i > 0:
                print("Average loss for this update:", round(update_loss/update_check,3))
                summary_update_loss.append(update_loss)
                
                # If the update loss is at a new minimum, save the model
                if update_loss <= min(summary_update_loss):
                    print('New Record!') 
                    stop_early = 0
                    saver = tf.train.Saver() 
                    saver.save(sess, checkpoint)

                else:
                    print("No Improvement.")
                    stop_early += 1
                    if stop_early == stop:
                        break
                update_loss = 0
            
                    
        # Reduce learning rate, but not below its minimum value
        learning_rate *= learning_rate_decay
        if learning_rate < min_learning_rate:
            learning_rate = min_learning_rate
        
        if stop_early == stop:
            print("Stopping Training.")
            break

[[ 0  0  0 ...,  0  0  0]
 [ 0  0  0 ...,  0  0  0]
 [ 0  0  0 ...,  0  0  0]
 [ 0  0  0 ...,  0  0 23]
 [ 0  0  0 ...,  0  0  0]]
[[ 0.  0.  0.  1.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.  0.]]


InvalidArgumentError: indices[3,999] = 23 is not in [0, 23)
	 [[Node: embedding_lookup = Gather[Tindices=DT_INT32, Tparams=DT_FLOAT, _class=["loc:@embedding_lookup/params_0"], validate_indices=true, _device="/job:localhost/replica:0/task:0/device:CPU:0"](embedding_lookup/params_0, _arg_input_0_0)]]

Caused by op u'embedding_lookup', defined at:
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/thomasan/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/thomasan/tensorflow/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/thomasan/tensorflow/lib/python2.7/site-packages/ipyke

In [ ]:
y_true = y_test.argmax(axis=1)
preds = model.predict_classes(X_test)
cm = ConfusionMatrix(y_true, preds)
cm.plot(backend='seaborn', normalized=True)
plt.title('Confusion Matrix Stars prediction')
plt.figure(figsize=(12, 10))